In [1]:
import spacy
from spacy import displacy
from spacy_conll import ConllFormatter
import os
import re
import xlsxwriter

In [2]:
nlp = spacy.load("en_core_web_lg")

In [3]:
conllformatter = ConllFormatter(nlp)
nlp.add_pipe(conllformatter, after='parser')

In [33]:
doc = nlp('And for this "Study in language variation : stress in Russian nouns" course paper, motivation comes from a changeable of language, which helps us to know how language modifies as time goes by.')
conll = doc._.conll_str
print(conll)

1	And	and	CCONJ	CC	ConjType=comp	19	cc	_	_
2	for	for	ADP	IN	_	19	prep	_	_
3	this	this	DET	DT	_	8	det	_	_
4	"	"	PUNCT	``	PunctType=quot|PunctSide=ini	8	punct	_	_
5	Study	Study	PROPN	NNP	NounType=prop|Number=sing	8	nmod	_	_
6	in	in	ADP	IN	_	5	prep	_	_
7	language	language	NOUN	NN	Number=sing	8	compound	_	_
8	variation	variation	NOUN	NN	Number=sing	2	pobj	_	_
9	:	:	PUNCT	:	_	8	punct	_	_
10	stress	stress	NOUN	NN	Number=sing	19	nsubj	_	_
11	in	in	ADP	IN	_	10	prep	_	_
12	Russian	russian	ADJ	JJ	Degree=pos	13	amod	_	_
13	nouns	noun	NOUN	NNS	Number=plur	11	pobj	_	_
14	"	"	PUNCT	''	PunctType=quot|PunctSide=fin	16	punct	_	_
15	course	course	NOUN	NN	Number=sing	16	compound	_	_
16	paper	paper	NOUN	NN	Number=sing	18	nmod	_	_
17	,	,	PUNCT	,	PunctType=comm	16	punct	_	_
18	motivation	motivation	NOUN	NN	Number=sing	19	nsubj	_	_
19	comes	come	VERB	VBZ	VerbForm=fin|Tense=pres|Number=sing|Person=three	0	ROOT	_	_
20	from	from	ADP	IN	_	19	prep	_	_
21	a	a	DET	DT	_	22	det	_	_
22	changeable	changeable	ADJ	JJ	Deg

In [11]:
def is_int(string):
    try:
        int(string)
        return True
    except Exception:
        return False

In [12]:
def open_file(filename):
    with open(filename, encoding='utf-8') as f:
        text = f.read()
    return text

In [34]:
def find_bad(words):
    erroneus = []
    bad_things = [
        ('NN', 'VBP'),
        ('NNS', 'VBZ'),
    ]
    text = []
    for key, value in words.items():
        text.append(value['text'])
    sent = ' '.join(text)
    pos_subj = ''
    pos_verb = ''
    predicates = []
    for number, char in words.items():
        if char['tag'] == 'ROOT':
            verb = char['text']
            pos_verb = char['pos']
            n = number
            predicates.append([verb, pos_verb, n])
    for number, char in words.items():
        if (char['tag'] == 'conj' or char['tag'] == 'ccomp') and char['s_pos'] == 'VERB':
            verb = char['text']
            pos_verb = char['pos']
            n = number
            predicates.append([verb, pos_verb, n])
    for number, char in words.items():
        if char['tag'] == 'acl':
            verb = char['text']
            pos_verb = char['pos']
            n = number
            predicates.append([verb, pos_verb, n])
    for pred in predicates:
        verb = pred[0]
        if verb.lower() in [
            'bet',
            'bid',
            'broadcast',
            'burst',
            'bust',
            'cast',
            'cost',
            'cut',
            'fit',
            'forecast',
            'hit',
            'hurt',
            'input',
            'knit',
            'let',
            'miswed',
            'preset',
            'put',
            'quit',
            'read',
            'rid',
            'set',
            'shed',
            'shut',
            'slit',
            'split',
            'spread',
            'sublet',
            'sweat',
            'thrust',
            'upset',
            'wed',
            'wet'
        ]:
            break
        pos_verb = pred[1]
        n = pred[2]
        for number, char in words.items():
            if char['dep'] == str(n) and char['tag'] == 'aux':
                pos_verb = char['pos']
                verb = char['text']
        sub = False
        countern = 0
        counterc = 0
        countera = 0
        for number, char in words.items():
            if char['dep'] == str(n) and (char['tag'] == 'nsubj' or char['tag'] == 'csubj'):
                subject = char['text']
                pos_subj = char['pos']
                if pos_subj == 'JJ':
                    return False
                k = number
                sub = True
                if char['tag'] == 'nsubj':
                    countern += 1
                elif char['tag'] == 'csubj':
                    counterc += 1
        if not sub:
            for number, char in words.items():
                if char['dep'] == str(n) and char['tag'] == 'attr':
                    for num, ch in words.items():
                        if ch['dep'] == str(n) and ch['tag'] == 'expl':
                            #print(sent)
                            sub = True
                            subject = char['text']
                            pos_subj = char['pos']
                            k = number
                            countera += 1
        if sub:
            if subject.lower() in [
                'bison',
                'cod',
                'deer',
                'fish',
                'moose',
                'offspring',
                'pike',
                'salmon',
                'sheep',
                'shrimp',
                'swine',
                'trout',
                'boar',
                'buffalo',
                'grouse',
                'elk',
                'fruit',
                'reindeer'
            ]:
                break
            if subject.lower() in 'government data statistics police team'.split():
                pos_subj = 'COLL'
            
            if subject in 'lot majority plenty part most group type class team half percent cent %'.split():
                for number, char in words.items():
                        if char['dep'] == str(k) and char['tag'] == 'prep':
                            if char['text'] == 'of':
                                order = number
                                for number, char in words.items():
                                    if char['dep'] == str(order) and char['tag'] == 'pobj':
                                        pos_subj = char['pos']
                                                
            if subject == 'number' or subject == 'lot':
                for number, char in words.items():
                    if char['dep'] == str(k) and char['tag'] == 'det':
                        if char['text'].lower() == 'a':
                            pos_subj = 'NNS'

            if subject in 'I i we We you You They they'.split():
                    pos_subj = 'NNS'
            elif subject in 'She she He he it It'.split():
                pos_subj = 'NN'

            for number, char in words.items():
                if char['dep'] == str(k) and char['tag'] == 'cc':
                    if char['text'].lower() == 'or':
                        for ka, ch in words.items():
                            if ch['dep'] == str(k) and ch['tag'] == 'conj':
                                subject = ch['text']
                                pos_subj = ch['pos']
                                if ch['text'] in 'I i we We you You They they'.split():
                                    pos_subj = 'NNS'
                                elif ch['text'] in 'She she He he it It'.split():
                                    pos_subj = 'NN'
                    else:
                        pos_subj = 'NNS'
                if char['dep'] == str(k) and char['tag'] == 'conj':
                    pos_subj = 'NNS'
            if countern > 1 or counterc > 1 or countera > 1:
                pos_subj = 'NNS'
            pattern = r'[\'"].*' + subject + r'.*[\'"].*' + verb
            if re.search(pattern, sent):
                pos_subj = 'NN'
                #print(sent)
            if (pos_subj, pos_verb) in bad_things:
                return pos_subj, pos_verb, subject, verb
    return False

In [36]:
def search(directory = 'C:/Users/anton/Рабочий стол/programming/awarl/new/'):
    errors = []
    folders = os.listdir(directory)
    #folders = ['2012-2014']
    for folder in folders:
        print(folder)
        files = os.listdir(directory + folder)
        for file in files:
            text = open_file(directory + folder + '/' + file)
            sentences = text.split('@')
            for sent in sentences:
                sent = re.sub(r'<.*>', '', sent)
                sent = re.sub(r' +', r' ', sent)
                sent = sent.replace('+', '')
                sent = sent.replace('*', '')
                #if 'there is' in sent.lower():
                    #print(sent)
                doc = nlp(sent)
                conll = doc._.conll_str
                lines = conll.splitlines()
                words = {}
                for line in lines:
                    word = line.split('\t')
                    if len(word) == 10:
                        # print(word)
                        words[int(word[0])] = {'text': word[1], 's_pos': word[3], 'pos': word[4], 'dep': word[6], 'tag': word[7]}
                try:
                    a = find_bad(words)
                    if a:
                        errors.append([
                            sent,
                            f'{a[2]} {a[3]}',
                            f'{a[0]}, {a[1]}',
                            file,
                            folder
                        ])
                except Exception:
                    print(sent)
    return errors

In [13]:
def writeln(errors, filename):
    workbook = xlsxwriter.Workbook(filename)
    worksheet = workbook.add_worksheet()
    row = 0
    col = 0
    for error in errors:
        worksheet.write(row, col, error[0])
        worksheet.write(row, col + 1, error[1])
        worksheet.write(row, col + 2, error[2])
        worksheet.write(row, col + 3, error[3])
        worksheet.write(row, col + 4, error[4])
        row += 1
    workbook.close()

In [37]:
a = search()
writeln(a, 'subj_verb_new.xlsx')

2012-2014
exam2014
exam2015
exam2016
exam2017
